In [ ]:
import sys 
sys.path.append("../..")
sys.path.append("..")

from importlib import reload

import joseph
from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *


reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)


model = HookedTransformer.from_pretrained(
    "gpt2-small",
    # "tiny-stories-2L-33M",
    # "attn-only-2l",
    # center_unembed=True,
    # center_writing_weights=True,
    # fold_ln=True,
    # refactor_factored_attn_matrices=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)




In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('jbloom/mats_sae_training_gpt2_small_resid_pre_5/sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152:v9', type='model')
artifact_dir = artifact.download()

In [ ]:

path = "./artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152:v28/1100001280_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_49152.pt"
# path = "/Users/josephbloom/GithubRepositories/mats_sae_training/artifacts/sparse_autoencoder_gelu-2l_blocks.0.hook_mlp_out_32768:v40/final_sparse_autoencoder_gelu-2l_blocks.0.hook_mlp_out_32768.pt"
# path = "./artifacts/sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152:v9/final_sparse_autoencoder_gpt2-small_blocks.5.hook_resid_pre_49152.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)

print(sparse_autoencoder.cfg)


# sanity check
text = "Many important transition points in the history of science have been moments when science 'zoomed in.' At these points, we develop a visualization or tool that allows us to see the world in a new level of detail, and a new field of science develops to study the world through this lens."
model(text, return_type="loss")

In [ ]:
from sae_training.utils import LMSparseAutoencoderSessionloader
model, sparse_autoencoder, activation_store = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
    path
)

# Sparsity

In [ ]:
from tqdm.auto import tqdm
def estimate_feature_sparsity_using_shuffled_activations(sparse_autoencoder, activation_store, n_batches):
    
    total_activations = torch.zeros(sparse_autoencoder.cfg.d_sae).to(sparse_autoencoder.cfg.device)
    
    pbar = tqdm(range(n_batches))
    for _ in pbar:
        # batch_tokens = activation_store.get_batch_tokens()
        # _, cache = model.run_with_cache(batch_tokens, prepend_bos=False)
        # original_act = cache[sparse_autoencoder.cfg.hook_point]
        original_act = activation_store.next_batch()
        _, feature_acts, _, _, _ = sparse_autoencoder(
            original_act
        )
        # for each batch item, pick 4 random tokens and keep only those
        # batch_size x n_tokens x d_sae
        # random_tok_indices = torch.randint(0, feature_acts.shape[1], (feature_acts.shape[0], 4))
        # feature_acts = feature_acts[torch.arange(feature_acts.shape[0]).unsqueeze(-1), random_tok_indices]
        total_activations += feature_acts.sum(0)
    
    total_tokens = (n_batches * feature_acts.shape[0]
                    )
    print("Total tokens:", total_tokens)
    
    return total_activations / total_tokens

In [ ]:
from tqdm.auto import tqdm
def estimate_feature_sparsity_using_n_features_per_prompt(
    sparse_autoencoder, activation_store, n_batches,
    n_tokens_per_prompt=4):
    
    total_activations = torch.zeros(sparse_autoencoder.cfg.d_sae).to(sparse_autoencoder.cfg.device)
    
    pbar = tqdm(range(n_batches))
    for _ in pbar:
        batch_tokens = activation_store.get_batch_tokens()
        _, cache = model.run_with_cache(batch_tokens, prepend_bos=False)
        original_act = cache[sparse_autoencoder.cfg.hook_point]
        _, feature_acts, _, _, _ = sparse_autoencoder(
            original_act
        )
        # for each batch item, pick 4 random tokens and keep only those
        # batch_size x n_tokens x d_sae
        random_tok_indices = torch.randint(0, feature_acts.shape[1], (feature_acts.shape[0], n_tokens_per_prompt))
        feature_acts = feature_acts[torch.arange(feature_acts.shape[0]).unsqueeze(-1), random_tok_indices]
        total_activations += feature_acts.flatten(0,1).sum(0)
    
    total_tokens = (n_batches * feature_acts.shape[0] * n_tokens_per_prompt)
    print("Total tokens:", total_tokens)
    
    return total_activations / total_tokens



n_prompts_per_batch, n_tokens_per_prompt = activation_store.get_batch_tokens().shape
log_feature_sparsities = []
n_dead_features_list = []
n_batches_list = [10, 100, 500]
n_tokens = [n_prompts_per_batch * n_tokens_per_prompt * i for i in n_batches_list]
for n_batches in tqdm(n_batches_list):
    feature_sparsity = estimate_feature_sparsity_using_n_features_per_prompt(sparse_autoencoder, activation_store, n_batches).detach().cpu()
    n_dead_features = (feature_sparsity == 0).sum()
    n_dead_features_list.append(n_dead_features.tolist())
    log_feature_sparsities.append(torch.log10(feature_sparsity + 1e-10).tolist())
    
tmp = pd.DataFrame(log_feature_sparsities, index=n_batches_list)


In [ ]:
px.histogram(
    tmp.T,#[tmp.T > -10],
    # color = tmp.columns,
    title="Feature Sparsity", 
    barmode="overlay",
    facet_col="variable",
    # log_y=True,
    labels={"value": "log10(sparsity)"})

In [ ]:
px.line((tmp.T ==  -10).sum() / tmp.shape[1]) 

In [ ]:
px.parallel_coordinates(
    tmp.T,
    color = tmp.columns,
    title="Feature Sparsity", 
    # range=[-10, 10],
    height=1000,
    labels={"value": "log10(sparsity)"})

In [ ]:
px.scatter(
    x = ((activation_store.storage_buffer > 0).sum(0).log10() + 1e-10).detach().cpu(),
    y = tmp.T.iloc[:,-1]
)

In [ ]:
px.imshow(tmp.T.corr().values, title="Feature Sparsity (320 prompts), 10 times", labels={"value": "log10(sparsity)"})

# Resampling Experiments

In [ ]:
global_loss_increases, global_input_activations = sparse_autoencoder.collect_anthropic_resampling_losses(
    model, activation_store
)

In [ ]:
sparse_autoencoder.cfg.resample_batches * sparse_autoencoder.cfg.store_batch_size

In [ ]:
global_loss_increases.shape

In [ ]:
px.histogram(global_loss_increases.flatten().detach().cpu(), log_y=False, title="Loss Increases")

In [ ]:
probs = global_loss_increases / global_loss_increases.sum()
px.histogram(probs.flatten().detach().cpu(), log_y=False, title="Loss Increases")

In [ ]:
import umap 
import hdbscan

reducer = umap.UMAP(
    n_neighbors=4,
    min_dist=0.15,
    n_components=2,
    metric="cosine",
    random_state=42,
)


ummap_result = reducer.fit_transform(global_input_activations.detach().cpu())



In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=10)
clusterer.fit(ummap_result)
print(clusterer.labels_)

fig = px.scatter(
        ummap_result, 
        x= 0,
        y=1,
        color=[str(i) for i in clusterer.labels_],
        opacity=0.5,
        template="plotly",
    )

# make points larger
# fig.update_traces(marker=dict(size=12))

# make it wide and tall
fig.update_layout(height=800, width=1200)
    
fig.show()



fig = px.scatter(
        ummap_result, 
        x= 0,
        y=1,
        color=global_loss_increases.flatten().detach().cpu(),
        # color_continuous_midpoint=0,
        color_continuous_scale="RdBu",
        opacity=0.5,
        template="plotly",
    )

# make points larger
# fig.update_traces(marker=dict(size=12))

# make it wide and tall
fig.update_layout(height=800, width=1200)
    
fig.show()

In [ ]:
range(clusterer.labels_.max()+1)

In [ ]:
tmp = pd.DataFrame(
    {
        "global_loss_increases": global_loss_increases.flatten().detach().cpu(),
        "cluster": clusterer.labels_,
    }
)
cluster_df = pd.DataFrame(
    {
        "cluster": range(clusterer.labels_.max()+2),
        "n_points": list(tmp.groupby("cluster").size().values),
        "mean_loss_increase": tmp.groupby("cluster").mean().values.T[0],
        "std": tmp.groupby("cluster").std().values.T[0],
    }
)
cluster_df.sort_values("mean_loss_increase", ascending=False).head(10).style.background_gradient(
    cmap="RdBu", subset=["mean_loss_increase"]
)


In [ ]:
group_specific_products = global_input_activations.cpu()[clusterer.labels_ == 298,] @ sparse_autoencoder.W_dec.T.detach().cpu()
px.histogram(group_specific_products.T, barmode="overlay", title="Group Specific Products")

In [ ]:
clusterer.labels_.shape

In [ ]:
global_input_activations

In [ ]:
test = (global_input_activations @ sparse_autoencoder.W_dec.T)
